[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
import time

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={}

# Assigning the name of the data to a variable
# The data within this file consists of all the data regarding hospital admissions
# and ventilator bed usage within England. 
# This is based on when the data was first recorded, all through to the most recent recorded data
# of the time that this JSON file was created.
file = "admission_ventilators.json"

# Reads the json file, and assigns the file to the variable 'data'
with open(file, "rt") as dataFile:
   data = json.load(dataFile)

# Adds the file name to the dictionary as the key, with the json file data as the value
jsondata[file] = data 


## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [4]:

# This function will sort the raw data into a DataFrame
def wrangle_data(rawdata):

    # Extracting the data from the file/Removing any information outside of the data itself
    dataCleaned = jsondata[file]['data']

    # Extracting the dates to be utilized for each index/row
    dates = [i['date'] for i in dataCleaned]
    
    # The dates needed to be sorted in ascending order, as they were received in descending order
    dates.sort()

    # assigning the start and end dates to variables
    startDate = pd.to_datetime(dates[0])
    endDate = pd.to_datetime(dates[-1])

    # Creating the range of dates to be assigned to the index of the dataframe. Range of dates set to daily.
    # Duplicate and missing dates will be resolved as every day within this range will be included in the index.
    index = pd.date_range(startDate, endDate, freq='D')


    # Constructing the DataFrame with 3 columns regarding admissions and ventilator bed usage.
    # Dates set as the indexes/rows
    adm_vent_df=pd.DataFrame(index=index, columns=['Admissions', 'Ventilator_Usage'])
    
    for i in dataCleaned:
        # Converting the date string of each line into a pandas datetime object
        date = pd.to_datetime(i['date'])
        
        # Replacing missing values or NaN (Not a Number) values with 0 for each column per date
        for column in ['Admissions', 'Ventilator_Usage']:
            if pd.isna(adm_vent_df.loc[date, column]):
                # Where the value of 0 is created to replace any missing values
                value = float(i[column]) if i[column] != None else 0.0
                # Assigning this value to the location of where the value is missing
                adm_vent_df.loc[date, column] = value

    # Replacing missing values/NAN for all the dates that were not accounted for in the above loop
    # Aka the dates missing from the original 'admission_ventilators.json file
    adm_vent_df.fillna(0.0, inplace=True)

    # Returns the DataFrame 
    return adm_vent_df



# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [5]:
# This function will download all the data from Public Health England regarding 
# admissions into hospital and ventilator usage throughout the country.
# The data received starts from when the data was first recorded, all through to the
# most recent record that is available to be downloaded.
def access_api():
    # Creates the dictionary of which the new raw data will be stored inn
    new_data = {}
    
    new_data_name = 'refreshed_adm_vent'
    
    # Filters the data received to the country 'England' in response to the request
    filter = ['areaType=nation', 'areaName=England' ]
    
    # Provides the order of data/structure and metrics to be requested from the coronavirus data
    structure = {'date': 'date',
            'Admissions': 'newAdmissions',
            'Ventilator_Usage': 'covidOccupiedMVBeds'}
    
    # Constructor for the Cov19API object with the filter and structure assigned
    api = Cov19API(filters=filter, structure=structure)
    
    # Retrieves the data from the API in JSON format 
    refreshed_adm_vent = api.get_json()

    # Assigns the new new_data_name to its data values in the new_data dictionary
    new_data[new_data_name] = refreshed_adm_vent 


    # Returns the new_data dictionary
    return new_data # return data read from the API


In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button

# 
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.

    # Assigns the access_api() function to a variable, where the output of the function is stored.
    refreshed_data=access_api()
    
    # wrangles the data and overwrites the dataframe for plotting
    global df
    df=wrangle_data(refreshed_data)

    # Sets button icon as 'check' sign once clicked and removes the button style.
    # Then stops any further executions for 4 seconds. Leaving the button disablled in this time.
    # Once 4 seconds is over, button is available again with the 'refresh' sign assigned to it.
    apiButton.icon="check"
    apiButton.disabled=True
    apiButton.description='Downloading...'
    apiButton.button_style=''
    time.sleep(4)
    apiButton.disabled=False
    apiButton.description='Refresh data'
    apiButton.button_style='info'
    apiButton.icon="refresh"
    
    # Calls the refresh_graph() function 
    refresh_graph()
    

# sets the parameters of the button
apiButton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info',
    tooltip='Click to download the current Public Health England Covid-19 data',
    icon='refresh')


# remember to register your button callback function with the button
apiButton.on_click(api_button_callback) # the name of your function inside these brackets

display(apiButton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click to…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [7]:
# Creates a widget that allows users to select the specific data they want to see.
# Data is either the hospital admissions data or the ventilator usage data.
# Users are able to select multiple pieces of data at the same time.
# Widgets - https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#numeric-widgets
selection = wdg.SelectMultiple(
    options = ['Admissions', 'Ventilator_Usage'],
    value = ['Admissions', 'Ventilator_Usage'],
    rows = 2,
    description = 'Stats:',
    disabled = False)

# Creates buttons that allow users to switch between graphs
# Graph options are either: line or bar
table_options = wdg.ToggleButtons(
    options=['Line', 'Bar (Long Wait Time)'],
    description='Graph Type:',
    disabled=False,
    button_style='',
    tooltips=['Line Graph', 'Bar Chart (Large data load)'])

# Variable counting the number of instances in the DataFrame
# For usage of the slider  to set the minimum and max indexs of the DataFrame
value=[x for x in range(len(df.index))]
# Creates a slider to specify the dates for the bar chart
slider = wdg.IntRangeSlider(
    value=(value[0], value[-1]),
    min=value[0],
    max=value[-1],
    step=1,
    description='Date Range:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d')

# Function that allows user to interact with graph options from the widgets
# Date_range is assigned to the values of the slider widget
def adm_vent_graph(columns, style, date_range):
    ncols = len(columns)
    if ncols > 0:
        # outputs a line graph if user selects line on the ToggleButtons widget
        if style == 'Line':
            # iloc[] is utilized with the date range in the index values 
            # to only show specific positions of the graph.
            # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html
            start, end = date_range
            date_range_df = df.iloc[start:end + 1]
            date_range_df[list(columns)].plot(kind='line')
            plt.title('Admissions/Ventilator Usage Line Graph')
            plt.xlabel('Dates')
            plt.ylabel('Occurences')
        # outputs a bar chart if user selects the bar chart option on the ToggleButtons widget   
        # Utilizes the IntRangeSlider widget to specify the date range for the graph
        elif style == 'Bar (Long Wait Time)':
            start, end = date_range
            date_range_df = df.iloc[start:end + 1]
            date_range_df[list(columns)].plot(kind='bar')
            plt.title('Admissions/Ventilator Usage Bar Chart')
            plt.xlabel('Dates')
            plt.ylabel('Occurences')
        plt.show()
    else:
        print('Select the data for graph')

    # Function that forces the graph the refresh when called.
    # Is based on the options of the ToggleButtons widget.
    def refresh_graph():
        current = table_options.options
        if current == table_options.options[0]:
            other = table_options.options[1]
        else:
            other = table_options.options[0]
        # Selects the button that isn't currently selected then selects back the currently selected button    
        table_options.options = other
        table_options.options = current


# Allows for interaction between the user, widgets and the graph, assigns this to a variable
# Data regarding hospital admissions and/or ventilator usage is expressed through the 
# form of either a line graph, or a bar chart.
#  Users are able to adjust the date range to be displayed within the graph.
graph = wdg.interactive_output(adm_vent_graph, {'columns': selection, 'style': table_options, 'date_range': slider})
# Creates a vertical box container that holds/organises widgets vertically, assigns this to a variable
controls=wdg.VBox([graph, selection, table_options, slider])
# Displays all widgets and graphs assigned to controls
display(controls)

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).